Randomモジュールをインポート

In [533]:
import random

### ネイピア数
$$e = (1+1/1000000000)^{1000000000}$$

In [534]:
def napiers_logarithm(x):
    return (1 + 1 / x) ** x
napier_number = napiers_logarithm(1000000000)  # e

### シグモイド関数
$$Sigmoid(x)=\frac{1}{1+Napiers(1000000000)^{-x}}$$

In [535]:
def sigmoid(x):
    return 1 / (1 + napier_number ** -x)

### シグモイド関数の微分
$$Sigmoid'(x)=Sigmoid(x)(1 - Sigmoid(x))$$

In [536]:
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

### ReLU関数
$$ReLU(x) =
        \begin{cases}
            x \quad x \geqq 0 \\
            0 \quad x < 0 \\
        \end{cases}
$$

In [537]:
def relu(x):
    return max(0, x)

### ReLU関数の微分
$$ReLU'(x) =
        \begin{cases}
            1 \quad x > 0 \\
            0 \quad x \leqq 0 \\
        \end{cases}
$$

In [538]:
def relu_derivative(x):
    return 1 if x > 0 else 0

### 自然対数
$$ln(x) = 2 \sum_{n=1,3,5,\dots}^{\infty} \frac{(z^n)}{n}, \quad z = \frac{x-1}{x+1}$$

In [539]:
def ln(x, n_terms=100):
    if x <= 0: raise ValueError("x must be positive")
    return (x - 1) / (x + 1) * sum([((x - 1) / (x + 1) ** n) / n for n in range(1, n_terms + 1, 2)])

### クロスエントロピー損失
$$L(y_{\text{true}}, y_{\text{pred}}) = - \sum_{i=1}^{n} y_{\text{true}_i} \cdot \ln(y_{\text{pred}_i} + \epsilon)$$

In [540]:
def cross_entropy_loss(y_true, y_pred):
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return -sum([t * ln(p + 1e-9) for t, p in zip(y_true, y_pred)])

### ニューラルネットワークを初期化
例:&emsp;入力層 -> 2,&emsp;隠れ層 -> [3, 3],&emsp;出力層 -> 1
#### 重み
$$n_{n}={2, 3, 3, 1}$$
$$ random \in [-1, 1] $$
$$[[[random]*n_{1}]*n_{2}, [[random]*n_{2}]*n_{3}, [[random]*n_{3}]*n_{4}]$$
#### バイアス
$$n_{n}={3, 3, 1}$$
$$ random \in [-1, 1] $$
$$[[random]*n_{1},[random]*n_{2},[random]*n_{3}]$$

※ [n]*5 -> [n, n, n, n, n]

In [541]:
def initialize_weights(layer_sizes):  # initialize weights and biases
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        weights.append([[random.uniform(-1, 1) for _ in range(layer_sizes[i])] for _ in range(layer_sizes[i+1])])  # [ {random_num *layer_sizes[i]} *layer_sizes[i+1] ]
        biases.append([random.uniform(-1, 1) for _ in range(layer_sizes[i+1])])  # [ random_num *layer_sizes[i+1] ]
    return weights, biases

### 順伝播処理

In [542]:
def forward_propagation(inputs, weights, biases):  # forward propagation
    activations = [inputs]
    for W, b in zip(weights, biases):
        activations.append([
            relu(sum([activations[-1][i] * W[j][i] for i in range(len(activations[-1]))]) + b[j])
            for j in range(len(b))
        ])
    return activations

### 逆伝播処理

In [543]:
def backward_propagation(activations, y_true, weights, biases, learning_rate):  # backward propagation
    output_layer = activations[-1]
    deltas = [[
        (output_layer[i] - y_true[i]) * sigmoid_derivative(output_layer[i])
        for i in range(len(y_true))
    ]]
    # Backpropagating the hidden layer error
    for l in range(len(weights)-1, 0, -1):
        deltas.insert(0, [
            sum([deltas[0][k] * weights[l][k][j] for k in range(len(deltas[0]))]) * relu_derivative(activations[l][j])
            for j in range(len(activations[l]))
        ])
    # Update the weights and biases
    for l in range(len(weights)):
        for i in range(len(weights[l])):
            for j in range(len(weights[l][i])):
                weights[l][i][j] -= learning_rate * deltas[l][i] * activations[l][j]
            biases[l][i] -= learning_rate * deltas[l][i]

    return weights, biases

### ニューラルネットワークを学習

In [544]:
def train(X, y, layer_sizes, epochs, learning_rate):
    weights, biases = initialize_weights(layer_sizes)
    weights_list = []
    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(X)):
            activations = forward_propagation(X[i], weights, biases)
            total_loss += cross_entropy_loss(y[i], activations[-1])
            weights, biases = backward_propagation(activations, y[i], weights, biases, learning_rate)
            weights_list.append(weights)
        
        m = (epoch + (epochs // 20) - 1) // (epochs // 20)
        print(f"\rEpoch {epoch+1}/{epochs}, Loss: {total_loss/len(X)}, [{'+'*m}{' '*(20-m)}]{' '*10}",end="")
    print("\nComplete")
    return weights, biases, weights_list

### パラメーターの設定および関数呼び出し

In [545]:
# DataSet for XOR
X = [[0, 0], [0, 1], [1, 0], [1, 1]]  # Input
y = [[0], [1], [1], [0]]  # Output

epochs = 500  # Number of epochs
learning_rate = 0.1  # Learning rate
layer_sizes = [2, 8, 16, 8, 1]  # 2 input -> 8 hidden -> 16 hidden -> 8 hidden -> 1 output

weights, biases, weights_list = train(X, y, layer_sizes, epochs, learning_rate)

for i in range(len(X)):  # Prediction
    activations = forward_propagation(X[i], weights, biases)
    output = activations[-1]
    binary_output = [1 if o >= 0.5 else 0 for o in output]
    print(f"Inputs: {X[i]}, Output: {y[i]} Predict: {binary_output}, probability: {round(output[0], 3)}")

Epoch 50000/50000, Loss: -1.373265586564151e-19, [++++++++++++++++++++]          
Complete
Inputs: [0, 0], Output: [0] Predict: [0], probability: 0
Inputs: [0, 1], Output: [1] Predict: [1], probability: 1.0
Inputs: [1, 0], Output: [1] Predict: [1], probability: 1.0
Inputs: [1, 1], Output: [0] Predict: [0], probability: 0


In [546]:
import numpy as np
import cv2

def normalize_weights(weights):
    """weights を -1～1 から 0～255 に変換し、uint8形式で返す"""
    weights = np.array(weights)  # リストをNumPy配列に変換
    return ((weights + 1) * 128).astype(np.uint8)  # 0～255に正規化

def create_frame_from_weights(current_weights, previous_weights, layer_sizes, frame_width, pixel_size=10):
    """各エポックごとに重みの変化を可視化したフレームを生成"""
    max_neurons = max(layer_sizes)  # 縦のサイズ（最大ニューロン数）
    frame = np.ones((max_neurons * pixel_size, frame_width * pixel_size, 3), dtype=np.uint8) * 255  # 白背景のフレームを作成

    x_pos = 0
    for layer_index, (current_layer_weights, previous_layer_weights) in enumerate(zip(current_weights, previous_weights)):
        # 重みの変化を計算し、10000倍にスケーリング
        weight_change = np.abs(np.array(current_layer_weights) - np.array(previous_layer_weights)) * 10000  # 重みの変化量
        normalized_weights = normalize_weights(weight_change)  # 重みの変化量を正規化
        
        # 色を付けるためにカラーマップを使用
        color_weights = cv2.applyColorMap(normalized_weights, cv2.COLORMAP_JET)  # カラーマップを適用
        layer_height, layer_width, _ = color_weights.shape

        # フレームにレイヤーの重みの変化をピクセルサイズで拡大表示
        for i in range(layer_height):
            for j in range(layer_width):
                if x_pos + j < frame_width and i < max_neurons:
                    # ピクセルサイズ分だけ重みを埋める（くっきり表示）
                    frame[i * pixel_size:(i + 1) * pixel_size, 
                          (x_pos + j) * pixel_size:(x_pos + j + 1) * pixel_size] = color_weights[i, j]
        x_pos += layer_width  # 次のレイヤーの配置位置を設定

    return frame

def weights_to_video(weights_list, layer_sizes, video_filename="weights_video.mp4", pixel_size=10):
    """weights_list をエポックごとにフレームとして動画を作成"""
    max_neurons = max(layer_sizes)  # 縦幅（最大ニューロン数）
    frame_width = sum(layer_sizes[:-1])  # 横幅（全レイヤーの合計ニューロン数）

    # 動画出力設定（フレームレート 10 FPS、各ピクセルがくっきり見えるように調整）
    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 10,  # フレームレートを10に設定
                          (frame_width * pixel_size, max_neurons * pixel_size), isColor=True)

    previous_weights = weights_list[0]  # 最初のエポックの重みを設定

    # 各エポックの重みをフレームに変換して書き込み
    for epoch_index, current_weights in enumerate(weights_list):
        if epoch_index > 0:
            frame = create_frame_from_weights(current_weights, previous_weights, layer_sizes, frame_width, pixel_size=pixel_size)
            out.write(frame)
            print(f"\rProcessing epoch {epoch_index}/{len(weights_list) - 1}", end="")
        previous_weights = current_weights  # 現在の重みを次のエポックの前の重みに設定

    out.release()
    print("\n動画作成が完了しました。")


In [547]:
# 使用例
weights_to_video(weights_list, layer_sizes, video_filename="weights_video.mp4")


Processing epoch 31/199999

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing epoch 199999/199999
動画作成が完了しました。
